**Se procede a importar las variables de entorno y almacenarlas en variables globales**

In [61]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
from datetime import datetime
load_dotenv()

True

**se definen a su vez las variables que se emplearan como parametros**

In [62]:
API_KEY = os.getenv("API_KEY")
API_URL= os.getenv("API_URL")
LANGUAGE = os.getenv("LANGUAGE","es-ES")
API_VERSION = os.getenv("API_VERSION",'3')
maximum_release_date = datetime.now().strftime("%Y-%m-%d")
minimum_release_date = "2020-10-15"

**Se realiza una request a la api de the movie database**

NOTA: puesto que la api recupera la información de manera paginada, se procedio a hacer un ciclo for para disparar la request en un total de 10 veces, y de esa manera, para  cada pagina recuperada se cargaba en el dataframe final (df)

In [63]:
num_pages = 10
df = pd.DataFrame()
for page in range(1, num_pages + 1):
    request = f'{API_URL}/{API_VERSION}/movie/now_playing?api_key={API_KEY}&language={LANGUAGE}&primary_release_date.gte={maximum_release_date}&primary_release_date.lte={minimum_release_date}&page={page}'
    response = requests.get(request)

    if response.status_code == 200:
        data = response.json()
        results = data.get('results', [])
        temp_df = pd.DataFrame(results)
        columns_to_drop = ["backdrop_path", "id", "original_language", "poster_path", "title", "video", "genre_ids"]
        temp_df.drop(columns=columns_to_drop, inplace=True)
        df = pd.concat([df, temp_df], ignore_index=True)
    else:
        print(f'Error en la solicitud de la página {page} => {response.status_code}')


**Se muestra los datos obtenidos en un dataframe**

In [64]:
df

,adult,original_title,overview,popularity,release_date,vote_average,vote_count
0,False,Five Nights at Freddy's,Un guardia de seguridad con problemas comienza...,7124.811,2023-10-25,8.472,927
1,False,Expend4bles,Cuarta entrega de la saga Los Mercenarios. El ...,1753.562,2023-09-15,6.510,522
2,False,Sound of Freedom,"Narra la historia de Tim Ballard, un exagente ...",1231.096,2023-07-03,8.130,1052
3,False,Gran Turismo,Un jugador adolescente de Gran Turismo gana un...,1536.966,2023-08-09,8.027,1227
4,False,The Equalizer 3,Desde que renunció a su vida como asesino del ...,1881.143,2023-08-30,7.344,1218
...,...,...,...,...,...,...,...
195,False,Ghost Adventures: Devil's Den,Una fuerza paranormal se percibe en un centro ...,13.560,2023-10-03,10.000,1
196,False,12th Fail,,16.375,2023-10-27,8.000,1
197,False,Annie Colère,"En 1973, en Francia, el Movimiento por la Lega...",15.653,2022-11-30,7.872,94
198,False,Puppy Love,"Después de una primera cita desastrosa, la niñ...",15.616,2023-08-22,7.038,53
